## libraries

In [ ]:
# Libraries
import tweepy
import json
import pandas as pd
pd.set_option('display.max_colwidth', -1) # to avoid truncated data
from pandas.io.json import json_normalize 
import datetime

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """


In [ ]:
import matplotlib.pyplot as plt
from textblob import TextBlob
import re
import numpy as np
import string

from wordcloud import WordCloud, STOPWORDS
from PIL import Image
from nltk.sentiment.vader import SentimentIntensityAnalyzer
!pip install langdetect
from langdetect import detect
from nltk.stem import SnowballStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer

/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [ ]:
import nltk
nltk.download('vader_lexicon')
nltk.download('stopwords')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

## get data

In [ ]:
# Keys for the request


In [ ]:
auth = tweepy.OAuthHandler(consumer_key,consumer_secret)
auth.set_access_token(access_token, access_token_secret)

In [ ]:
api = tweepy.API(auth, wait_on_rate_limit = True, wait_on_rate_limit_notify = True)

In [ ]:
data = api.me()

In [ ]:
# function to convert _json to JSON
def jsonify_tweepy(tweepy_object):
    json_str = json.dumps(tweepy_object._json)
    return json.loads(json_str)

# function to get extended retweets
def jsonify_tweepy_2(tweet_info):

    if "retweeted_status" in dir(tweet_info):
      return tweet_info.retweeted_status.full_text
    else:
      return tweet_info.full_text

# df.iloc[row][col]

In [ ]:
# searching by keywords
keyword_res = list(tweepy.Cursor(api.search, q="mustang mach e", tweet_mode="extended").items(3000))
keyword_df = json_normalize([jsonify_tweepy(tweet) for tweet in keyword_res])
keyword_df.full_text = pd.DataFrame([jsonify_tweepy_2(tweet) for tweet in keyword_res ])
print(len(keyword_df))

In [ ]:
#keyword_df.head()

In [ ]:
keyword_df.to_excel("mustang_mach_e041022.xlsx")

## NLP processing

In [ ]:
keyword_df = pd.read_excel("mustang_mach_e041022.xlsx")
tweets = keyword_df.full_text

In [ ]:
def percentage(part,whole):
    return 100 * float(part)/float(whole)

In [ ]:
tweet_list = pd.DataFrame([tweet for tweet in tweets])
tweet_list.drop_duplicates(inplace = True)

In [ ]:
tw_list = pd.DataFrame(tweet_list)
tw_list["text"] = tw_list[0]
tw_list.head()

,0,text
0,@Jaswht Tesla? Mustang mach e?,@Jaswht Tesla? Mustang mach e?
1,Ford Mustang Mach E 1400 🔥\n#ForzaHorizon5 @ForzaHorizon https://t.co/UfsOrEaLYo,Ford Mustang Mach E 1400 🔥\n#ForzaHorizon5 @ForzaHorizon https://t.co/UfsOrEaLYo
4,https://t.co/F80hQJ46M8,https://t.co/F80hQJ46M8
5,@naimsharif0 Mustang Mach E,@naimsharif0 Mustang Mach E
6,@CarlKoinberg @sdmoores @benchmarkmin Yes. Mustang Mach e battery size is 98kwh and e-tron is 95kwh .... Absurd.,@CarlKoinberg @sdmoores @benchmarkmin Yes. Mustang Mach e battery size is 98kwh and e-tron is 95kwh .... Absurd.


In [ ]:
#Creating new dataframe and new features
tw_list = pd.DataFrame(tweet_list)
tw_list["text"] = tw_list[0]

#Removing RT, Punctuation etc
remove_rt = lambda x: re.sub('RT @\w+: '," ",x)
rt = lambda x: re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",x)
tw_list["text"] = tw_list.text.map(remove_rt).map(rt)
tw_list["text"] = tw_list.text.str.lower()
tw_list.head()

,0,text
0,@Jaswht Tesla? Mustang mach e?,tesla mustang mach e
1,Ford Mustang Mach E 1400 🔥\n#ForzaHorizon5 @ForzaHorizon https://t.co/UfsOrEaLYo,ford mustang mach e 1400 forzahorizon5
4,https://t.co/F80hQJ46M8,
5,@naimsharif0 Mustang Mach E,mustang mach e
6,@CarlKoinberg @sdmoores @benchmarkmin Yes. Mustang Mach e battery size is 98kwh and e-tron is 95kwh .... Absurd.,yes mustang mach e battery size is 98kwh and e tron is 95kwh absurd


In [ ]:
tw_list[['polarity', 'subjectivity']] = tw_list['text'].apply(lambda Text: pd.Series(TextBlob(Text).sentiment))
for index, row in tw_list['text'].iteritems():
    score = SentimentIntensityAnalyzer().polarity_scores(row)
    neg = score['neg']
    neu = score['neu']
    pos = score['pos']
    comp = score['compound']
    if neg > pos:
        tw_list.loc[index, 'sentiment'] = "negative"
    elif pos > neg:
        tw_list.loc[index, 'sentiment'] = "positive"
    else:
        tw_list.loc[index, 'sentiment'] = "neutral"
    tw_list.loc[index, 'neg'] = neg
    tw_list.loc[index, 'neu'] = neu
    tw_list.loc[index, 'pos'] = pos
    tw_list.loc[index, 'compound'] = comp

tw_list.head(3)

,0,text,polarity,subjectivity,sentiment,neg,neu,pos,compound
0,@Jaswht Tesla? Mustang mach e?,tesla mustang mach e,0.0,0.0,neutral,0.0,1.0,0.0,0.0
1,Ford Mustang Mach E 1400 🔥\n#ForzaHorizon5 @ForzaHorizon https://t.co/UfsOrEaLYo,ford mustang mach e 1400 forzahorizon5,0.0,0.0,neutral,0.0,1.0,0.0,0.0
4,https://t.co/F80hQJ46M8,,0.0,0.0,neutral,0.0,0.0,0.0,0.0


In [ ]:
#Creating new data frames for all sentiments (positive, negative and neutral)

tw_list_negative = tw_list[tw_list["sentiment"]=="negative"]
tw_list_positive = tw_list[tw_list["sentiment"]=="positive"]
tw_list_neutral = tw_list[tw_list["sentiment"]=="neutral"]

In [ ]:
#Function for count_values_in single columns

def count_values_in_column(data,feature):
    total=data.loc[:,feature].value_counts(dropna=False)
    percentage=round(data.loc[:,feature].value_counts(dropna=False,normalize=True)*100,2)
    return pd.concat([total,percentage],axis=1,keys=['Total','Percentage'])

In [ ]:
#Count_values for sentiment
count_values_in_column(tw_list,"sentiment")

,Total,Percentage
positive,115,45.28
neutral,97,38.19
negative,42,16.54


In [ ]:
tw_list_negative.head(10)

,0,text,polarity,subjectivity,sentiment,neg,neu,pos,compound
11,Ford Mustang Mach-E Buyers Overwhelmingly Opting For AWD https://t.co/hrDd0YFfaS https://t.co/IWBgWxfSVH,ford mustang mach e buyers overwhelmingly opting for awd,0.500000,1.000000,negative,0.176,0.824,0.000,-0.1280
23,"@KeepinItGutta Ye but it’s worse battery, self driving tech, and safety rating. Roughly 4 mustang Mach e of battery materials makes 5 model 3/Y… etc etc",ye but it s worse battery self driving tech and safety rating roughly 4 mustang mach e of battery materials makes 5 model 3 y etc etc,-0.250000,0.500000,negative,0.155,0.708,0.138,-0.1154
41,"@technikblog_ch @vwschweiz @AMAG_Media @cariad_tech Habe jetzt 22 Tkm mit dem Mustang Mach e und freue mich immer noch, wie am 1. Tag.\nid4 wäre die Alternative gewesen, aber es ist für mich das Pony geworden.\nIm Sommer kommt der erste elektrische Familienurlaub...freue mich drauf.\nNie mehr Verbrenner.",ch media tech habe jetzt 22 tkm mit dem mustang mach e und freue mich immer noch wie am 1 tag id4 w re die alternative gewesen aber es ist f r mich das pony geworden im sommer kommt der erste elektrische familienurlaub freue mich drauf nie mehr verbrenner,0.000000,0.000000,negative,0.083,0.917,0.000,-0.5994
48,this group of teens just hopped out a mustang mach e. i wanna be rich so bad,this group of teens just hopped out a mustang mach e i wanna be rich so bad,-0.162500,0.708333,negative,0.224,0.597,0.179,-0.2243
73,🚘 #CarReviewRating I La nueva versión más deportiva Ford Mustang Mach-E GT de 487 CV demuestra sus sorprendentes prestaciones sin emisiones en nuestra prueba https://t.co/hA8cA8q09l https://t.co/SYwAgW3Ao2,carreviewrating i la nueva versi n m s deportiva ford mustang mach e gt de 487 cv demuestra sus sorprendentes prestaciones sin emisiones en nuestra prueba,0.000000,0.000000,negative,0.146,0.769,0.085,-0.3612
98,@TheCOAPodcast @chrishibbs13 @jessicai917 @DanielTurnerPTF Finally we do not have the infrastructure to support charging these cars and also time constraints. A Kia Ev6 can quick charge in 18 minutes to 80% vs the Mustang Mach E 45 min. If you are third in line you wait a hour and still have to charge. Haven’t even touched on our grid,finally we do not have the infrastructure to support charging these cars and also time constraints a kia ev6 can quick charge in 18 minutes to 80 vs the mustang mach e 45 min if you are third in line you wait a hour and still have to charge haven t even touched on our grid,0.111111,0.500000,negative,0.074,0.879,0.047,-0.3612
100,"@wasallesbeter Een oude aftandse @FordNL mondeo 2.0 automaat uit 2002\nHij rijd nog maar hij zuipt op korte afstanden naar mijn werk!\nIk heb al een mustang mach E gevraagd aan ford, maar daar beginnen ze niet aan! Dan moet ik maar in die ouwe agenebbis auto blijven rijden!",een oude aftandse mondeo 2 0 automaat uit 2002 hij rijd nog maar hij zuipt op korte afstanden naar mijn werk ik heb al een mustang mach e gevraagd aan ford maar daar beginnen ze niet aan dan moet ik maar in die ouwe agenebbis auto blijven rijden,0.000000,0.000000,negative,0.081,0.919,0.000,-0.5994
107,New Kia EV6 – better than a Mustang Mach-E? – EAST AUTO NEWS https://t.co/lMtkvxWXfh,new kia ev6 better than a mustang mach e east auto news,0.318182,0.477273,negative,0.278,0.530,0.192,-0.3182
108,New Kia EV6 – better than a Mustang Mach-E? https://t.co/7dZqxBuQrA,new kia ev6 better than a mustang mach e,0.318182,0.477273,negative,0.347,0.413,0.240,-0.3182
113,the amount of men who have seen the bronco logo in my snaps/stories and proceeded to ask if i drive a mustang/mach e absolutely kills me every time,the amount of men who have seen the bronco logo in my snaps stories and proceeded to ask if i drive a mustang mach e absolutely kills me every time,0.200000,0.900000,negative,0.127,0.873,0.000,-0.5849
